In [ ]:
from photoDownloader import *
import time
import re
import pandas as pd
API_URL = "http://localhost:3000/api/products"


In [5]:
file = pd.read_csv("listaGrande.csv")
# Columnas que queremos extraer
cols = [
    "Cód.",
    "Cód. Fab.",
    "Descripción",
    "Rubro",
    "Marca",
    "Stock",
    "Precios con IVA (DOLAR (U$S))",
    "Descripción Detallada",
    "Descripción Larga",
]

# Filtrar solo esas columnas
filtered = file[cols]

In [6]:
item = filtered.iloc[4]
x = str(f"{item["Marca"]} {item["Descripción"]}")[:20]
print(x)

Noganet Mouse optico


In [7]:
# Función para determinar categoría y markup basado en el análisis de precios
downloader = PhotoDownloader(headless=False)  # headless=False para que cargue bien las imágenes

def get_markup_category(price_usd):
    """
    Categoriza productos según el análisis realizado:
    - Categoría 1 (≤$6.36): 100% markup
    - Categoría 2 ($6.38-$13.55): 66% markup  
    - Categoría 3 ($13.69-$31.97): 40% markup
    - Categoría 4 (≥$32.03): 20% markup
    """
    if price_usd <= 6.36:
        return 1, 100.0
    elif price_usd <= 13.55:
        return 2, 75.0
    elif price_usd <= 31.97:
        return 3, 45.0
    else:
        return 4, 30.0

# Iterar sobre filas
for _, item in file.iterrows():
    try:
        # Procesar datos
        name = f"{item['Marca']} {item['Descripción']}"[:35]  # nombre corto
        description = str(item["Descripción Larga"]) if pd.notna(item["Descripción Larga"]) else str(item["Descripción"])
        
        # Precio original en USD
        price_usd = float(item["Precios con IVA (DOLAR (U$S))"])
        
        # Si el precio es 0, saltar este item
        if price_usd == 0:
            print(f"⏭️ Saltado por precio 0: {item['Descripción']}")
            continue
        
        # Determinar categoría y markup
        category_num, markup = get_markup_category(price_usd)
        
        # Calcular precio con markup aplicado
        price_with_markup = price_usd * (1 + markup/100)
        
        # Multiplicar por 1423 para precio final
        final_price = price_with_markup * 1423
        
        # Calcular basePrice (precio original sin markup)
        basePrice = price_usd * 1423

        stock = 1 if str(item["Stock"]).strip().lower() == "si" else 0
        category = str(item["Rubro"]) if pd.notna(item["Rubro"]) else "general"

        # Nombre imagen
        base_name = f"{item['Marca']} {item['Descripción']}"[:80]
        # Eliminar caracteres problemáticos que crean carpetas o rompen rutas
        base_name = base_name.replace("/", " ").replace("\\", " ")
        base_name = base_name.replace('"', "").replace("'", "")
        base_name = base_name.replace(":", " ").replace("|", " ")
        base_name = base_name.replace("*", " ").replace("?", " ")
        base_name = base_name.replace("<", " ").replace(">", " ")
        
        # Normalizar espacios a un solo espacio
        base_name = re.sub(r"\s+", " ", base_name).strip()
        
        # Reemplazar espacios por guión bajo y colapsar múltiples guiones bajos
        image_file_name = base_name.replace(" ", "_")
        image_file_name = re.sub(r"_+", "_", image_file_name)
        image_file_name = image_file_name.strip("._")  # evitar empezar/terminar con . o _
        
        # Limitar longitud y agregar extensión
        image_file_name = image_file_name[:60] + ".jpg"

        image_search_str = f"{item['Marca']} {item['Descripción']}"[:35]
        downloader.descargar_foto(image_search_str, image_file_name)

        # JSON a enviar
        payload = {
            "name": name,
            "description": description,
            "price": final_price,  # Precio final con markup y multiplicado por 1423
            "basePrice": basePrice,  # Precio base original multiplicado por 1423
            "markup": markup,  # Porcentaje de markup aplicado
            "stock": stock,
            "category": category,
            "image": image_file_name
        }

        # Enviar a API
        resp = requests.post(API_URL, json=payload)
        if resp.status_code == 201:
            print(f"✅ Producto creado: {name} | Cat.{category_num} | ${price_usd:.2f} → ${final_price:,.0f} (+{markup}%)")
        else:
            print(f"⚠️ Error {resp.status_code}: {resp.text}")
        print("Esperando 5 segundos")
        time.sleep(5)

    except Exception as e:
        print("❌ Error procesando fila:", e)
        print("Esperando 5 segundos")
        time.sleep(5)

[Generica 	 PC ARMADA -Intel Core i3] Imagen guardada en: /home/bartolo/Documents/it360/it360_Soluciones/public/images/Generica_PC_ARMADA_-Intel_Core_i3-12100_-3.30GHz_+_Gigabyte_.jpg
✅ Producto creado: Generica 	 PC ARMADA -Intel Core i3 | Cat.4 | $342.82 → $634,183 (+30.0%)
Esperando 5 segundos
⏭️ Saltado por precio 0: Adaptador minidin H/H -6 pines -gray -Nisuta (NS-ADPSHH)
[Hewlett Packard Impresora multifunc] Imagen guardada en: /home/bartolo/Documents/it360/it360_Soluciones/public/images/Hewlett_Packard_Impresora_multifuncion_-usb_2.0_bt_wifi_wifi.jpg
✅ Producto creado: Hewlett Packard Impresora multifunc | Cat.4 | $259.98 → $480,937 (+30.0%)
Esperando 5 segundos
[Netmak           Smartwatch Kids -0] Imagen guardada en: /home/bartolo/Documents/it360/it360_Soluciones/public/images/Netmak_Smartwatch_Kids_-0.96_oled_-bluetooh_4.0_-apk_android.jpg
✅ Producto creado: Netmak           Smartwatch Kids -0 | Cat.2 | $8.48 → $21,117 (+75.0%)
Esperando 5 segundos
[Noganet Mouse optico -usb 

KeyboardInterrupt: 